# Import python packages

In [1]:
import pandas as pd
import numpy as np
# %pip install pypandoc --user
# %pip install jupyslides --user
# %pip install yellowbrick --user
# %pip install plotly --user
# %pip install dill --user

In [2]:
import pandas as pd
import numpy as np

## Create database

In [4]:
import pyodbc
server = 'localhost\SQLEXPRESS' 
username = 'sa' 
password = 'user1' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';UID='+username+';PWD='+ password,autocommit=True)
cursor = cnxn.cursor()

sql1='''
USE master;
ALTER DATABASE DataWarehouse1 SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
DROP DATABASE IF EXISTS DataWarehouse1
create database DataWarehouse1

ALTER DATABASE DataWarehouse2 SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
DROP DATABASE IF EXISTS DataWarehouse2
create database DataWarehouse2

ALTER DATABASE datahub SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
DROP DATABASE IF EXISTS datahub
create database datahub


'''


cursor.execute(sql1)
cnxn.commit()

cursor.close()

# Establish connection to database

In [3]:
import pyodbc
import urllib
import sqlalchemy

'''sources databases'''
params_p = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=DataWarehouse1;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_p = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_p))


params_s = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=DataWarehouse2;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_s = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_s))



'''destination databases'''

params_datahub = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=datahub;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_datahub = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_datahub))

# Import data & Export data to database

In [8]:
import glob
from pathlib import Path

directoryPath=r'C:/Business_Data_Analysis/data/'
for file_name in glob.glob(directoryPath+'*.csv'):
    if 'sales' not in file_name:
        print(Path(file_name).stem)
        x = pd.read_csv(file_name)
        x.to_sql(Path(file_name).stem, engine_datahub,if_exists='replace',index=False)


ab_data
audit_engagement_data
audit_fraud_risk_data
audit_outlier_data
audit_post_data
audit_Q3_Data_30
audit_Q3_Data_31
audit_qualitative_bankruptcy
credit_score
credit_test
credit_train
HR_Employee_Attrition
marketing_churn
primary_data_funnel
secondary_data_funnel


In [12]:
'''sales data'''

'''initial ETL to database'''
df_sales_p=pd.read_csv(directoryPath+'sales_primary_data.csv',parse_dates=['onboarded_month','user_login_month','auth_month'],infer_datetime_format=True,index_col=False)
df_sales_s=pd.read_csv(directoryPath+'sales_secondary_data.csv',parse_dates=['user_login_month','auth_month'],infer_datetime_format=True,index_col=False)

'''export data to datahub'''
df_sales_s.to_sql("sales_data_secondary", engine_datahub, if_exists='replace',index=False)
df_sales_p.to_sql("sales_data_primary", engine_datahub,if_exists='replace',index=False)
# df_sales_p.to_sql("sql_sales_data_primary", engine_p,if_exists='replace',index=False)
df_sales_p.to_sql("sql_sales_data_primary", engine_datahub,if_exists='replace',index=False)


# Check data

In [10]:
# df_sales_p.shape